# Data Structures 

xarray has two core data structures. Both are fundamentally N-dimensional:

- DataArray is xarray's implementation of a labeled, N-dimensional array.
- Dataset is a multi-dimensional, in-memory array database. It is a dict-like container of DataArray objects aligned along any number of shared dimensions.

-----

### Outline
- Arrays of Numbers (Numpy is Python's most common array library)
- The common data model (labels, netcdf)
- N-Dimensional labeled arrays (xarray)
  - DataArray
  - Dataset
  
### Tutorial Duriation
10 minutes

### Going Further

Xarray Documentation on Data Structures: http://xarray.pydata.org/en/latest/data-structures.html

In [1]:
import numpy as np
import xarray as xr

## Arrays of Numbers

Unlabeled, N-dimensional arrays of numbers (e.g., NumPy’s ndarray) are the most widely used data structure in scientific computing. However, they lack a meaningful representation of the metadata associated with their data. Implementing such functionality is left to individual users and domain-specific packages. As a result, programmers frequently encounter pitfalls in the form of questions like “is the time axis of my array in the first or third index position?” or “does my array of timestamps still align with my data after resampling?”.

In [2]:
myvar = np.random.random(size=(2, 3, 6))
myvar

array([[[0.74566189, 0.33075956, 0.75066661, 0.61140909, 0.17996278,
         0.43540762],
        [0.42278927, 0.99823537, 0.07558881, 0.28165414, 0.20818678,
         0.77214002],
        [0.56374381, 0.66286714, 0.72359682, 0.82222223, 0.33898965,
         0.37934316]],

       [[0.05525711, 0.25420931, 0.59866917, 0.69592386, 0.40769198,
         0.57055577],
        [0.5316985 , 0.44711762, 0.0926392 , 0.39773454, 0.91456689,
         0.27576791],
        [0.20595209, 0.13857103, 0.44889883, 0.51192085, 0.35003589,
         0.57585388]]])

## The Common Data Model and Inspiration from NetCDF

![](images/dataset-diagram.png)
*An example of how a dataset (netCDF or xarray) for a weather forecast might be structured. This dataset has three dimensions, time, y, and x, each of which is also a one-dimensional coordinate. Temperature and precipitation are three-dimensional data variables. Also included in the dataset are two-dimensional coordinates latitude and longitude, having dimensions y and x, and reference time, a zero-dimensional (scalar) coordinate.*

xarray adopts Unidata’s self-describing Common Data Model on which the network Common Data Form (netCDF) is built [20, 7]. NetCDF provides a well-defined data model for labeled N-dimensional array-oriented scientific data analysis.

## Xarray Data Structures

![](images/xarray-data-structures.png)

The Common Data Model and NetCDF forms the basis of the xarray data model and provides a natural and portable serialization format. Building on netCDF, xarray features two main data structures: the DataArray and the Dataset. The API for these data structures is summarized in the following sections and in the figure above.

## `xarray.DataArray`

The DataArray is xarray’s implementation of a labeled, multi-dimensional array. It has several key properties:

- data: N-dimensional array (NumPy or dask) holding the array's values,
- coords: dict-like container of arrays (coordinates) that label each point (e.g., 1-dimensional arrays of numbers, datetime objects or strings),
- dims: dimension names for each axis [e.g., (‘time’, ‘latitude’, ‘longitude’)],
- attrs: OrderedDict holding arbitrary metadata (e.g. units or descriptions), and
- name: an arbitrary name for the array.

xarray uses dims and coords to enable its core metadata-aware operations. Dimensions provide names that xarray uses instead of the axis argument found in many NumPy functions. Coordinates are ancillary variables used to enable fast label based indexing and alignment, building on the functionality of the pandas Index. DataArray objects also can have a name and can hold arbitrary metadata in the form of their attrs property, which can be used to further describe data (e.g. by providing units). Names and attributes are strictly for users and user-written code; in general xarray makes no attempt to interpret them, and propagates them only in unambiguous cases. In contrast, xarray does interpret and persist coordinates in operations that transform xarray objects.



In [6]:
my_da = xr.DataArray(myvar)
my_da

<xarray.DataArray (dim_0: 2, dim_1: 3, dim_2: 6)>
array([[[0.745662, 0.33076 , 0.750667, 0.611409, 0.179963, 0.435408],
        [0.422789, 0.998235, 0.075589, 0.281654, 0.208187, 0.77214 ],
        [0.563744, 0.662867, 0.723597, 0.822222, 0.33899 , 0.379343]],

       [[0.055257, 0.254209, 0.598669, 0.695924, 0.407692, 0.570556],
        [0.531699, 0.447118, 0.092639, 0.397735, 0.914567, 0.275768],
        [0.205952, 0.138571, 0.448899, 0.511921, 0.350036, 0.575854]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

In [8]:
# Adding labels/metadata
my_da = xr.DataArray(myvar,
                     dims=('lat', 'lon', 'time'),
                     coords={'lat': [15., 30.], 'lon': [-110., -115., -120.]},
                     attrs={'long_name': 'temperature', 'units': 'C'},
                     name='temp')
my_da

<xarray.DataArray 'temp' (lat: 2, lon: 3, time: 6)>
array([[[0.745662, 0.33076 , 0.750667, 0.611409, 0.179963, 0.435408],
        [0.422789, 0.998235, 0.075589, 0.281654, 0.208187, 0.77214 ],
        [0.563744, 0.662867, 0.723597, 0.822222, 0.33899 , 0.379343]],

       [[0.055257, 0.254209, 0.598669, 0.695924, 0.407692, 0.570556],
        [0.531699, 0.447118, 0.092639, 0.397735, 0.914567, 0.275768],
        [0.205952, 0.138571, 0.448899, 0.511921, 0.350036, 0.575854]]])
Coordinates:
  * lat      (lat) float64 15.0 30.0
  * lon      (lon) float64 -110.0 -115.0 -120.0
Dimensions without coordinates: time
Attributes:
    long_name:  temperature
    units:      C

In [13]:
# The underlying data is still there:
print(type(my_da))
print(type(my_da.data))
print(my_da.attrs)
my_da.data

<class 'xarray.core.dataarray.DataArray'>
<class 'numpy.ndarray'>
OrderedDict([('long_name', 'temperature'), ('units', 'C')])


array([[[0.74566189, 0.33075956, 0.75066661, 0.61140909, 0.17996278,
         0.43540762],
        [0.42278927, 0.99823537, 0.07558881, 0.28165414, 0.20818678,
         0.77214002],
        [0.56374381, 0.66286714, 0.72359682, 0.82222223, 0.33898965,
         0.37934316]],

       [[0.05525711, 0.25420931, 0.59866917, 0.69592386, 0.40769198,
         0.57055577],
        [0.5316985 , 0.44711762, 0.0926392 , 0.39773454, 0.91456689,
         0.27576791],
        [0.20595209, 0.13857103, 0.44889883, 0.51192085, 0.35003589,
         0.57585388]]])

## `xarray.Dataset`

The Dataset is xarray’s multi-dimensional equivalent of a DataFrame. It is a dict-like container of labeled arrays (DataArrays) with aligned dimensions. It is designed as an in-memory representation of a netCDF dataset. In addition to the dict-like interface of the dataset itself, which can be used to access any DataArray in a Dataset, datasets have four key properties:

- data_vars: OrderedDict of DataArray objects corresponding to data variables,
- coords: OrderedDict of DataArray objects intended to label points used in data_vars (e.g., 1-dimensional arrays of numbers, datetime objects or strings),
- dims: dictionary mapping from dimension names to the fixed length of each dimension (e.g., {‘x’: 6, ‘y’: 6, ‘time’: 8}), and
- attrs: OrderedDict to hold arbitrary metadata pertaining to the dataset.
DataArray objects inside a Dataset may have any number of dimensions but are presumed to share a common coordinate system. Coordinates can also have any number of dimensions but denote constant/independent quantities, unlike the varying/dependent quantities that belong in data. Figure 3 illustrates these concepts for an example Dataset containing meteorological data.

In [15]:
# Datasets are dict-like containers of DataArrays
# Here's an empty dataset:
xr.Dataset()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*

In [16]:
my_ds = xr.Dataset({'temperature': my_da})
# also equivalent to:
# my_da.to_dataset()
my_ds

<xarray.Dataset>
Dimensions:      (lat: 2, lon: 3, time: 6)
Coordinates:
  * lat          (lat) float64 15.0 30.0
  * lon          (lon) float64 -110.0 -115.0 -120.0
Dimensions without coordinates: time
Data variables:
    temperature  (lat, lon, time) float64 0.7457 0.3308 0.7507 0.6114 0.18 ...

In [17]:
my_ds['precipitation'] = xr.DataArray(np.random.random(myvar.shape),
                                      dims=('lat', 'lon', 'time'),
                                      coords={'lat': [15., 30.], 'lon': [-110., -115., -120.]},
                                      attrs={'long_name': 'precipitation', 'units': 'mm'},
                                      name='pcp') 

my_ds.attrs['history'] = 'created for the xarray tutorial'

In [18]:
my_ds

<xarray.Dataset>
Dimensions:        (lat: 2, lon: 3, time: 6)
Coordinates:
  * lat            (lat) float64 15.0 30.0
  * lon            (lon) float64 -110.0 -115.0 -120.0
Dimensions without coordinates: time
Data variables:
    temperature    (lat, lon, time) float64 0.7457 0.3308 0.7507 0.6114 0.18 ...
    precipitation  (lat, lon, time) float64 0.2395 0.3941 0.8607 0.7073 ...
Attributes:
    history:  created for the xarray tutorial